In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, Activation, Dropout, GRU, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras import activations
from tensorflow.keras.layers import Layer
from tensorflow.keras import layers
import tensorflow as tf

In [2]:
from Seq2Seq import clayers
from Seq2Seq import cactivations

In [3]:
from tensorflow.keras.layers import GRU, concatenate, Lambda

In [4]:
tf.__version__

'1.14.0'

In [5]:
ENCODER_SEQ_LEN = 30
DECODER_SEQ_LEN = 20
VOCAB_SIZE = 500
units = 16

In [6]:
x = np.random.randint(0, 499, size=(2000, ENCODER_SEQ_LEN))
y = np.random.randint(0, 499, size=(2000, DECODER_SEQ_LEN))

In [7]:
tf.keras.backend.clear_session()
def encoder():
    input_encoder = Input(shape=(ENCODER_SEQ_LEN,),name='encoder_input')

    x_embedded = Embedding(input_dim=VOCAB_SIZE, output_dim=50, input_length=ENCODER_SEQ_LEN,
                           mask_zero=False, name="embedding_layer_encoder")(input_encoder)
    
    gru_output, gru_state= GRU(units, return_state=True, return_sequences=True, name="Encoder_GRU")(x_embedded)
    return Model(inputs=input_encoder, outputs=[gru_output, gru_state])

In [8]:
def OneStepDecoder():
    
    #inputs
    input_decoder = Input(shape=(1,),name="dec_input")
    
    input_state = Input(shape=(units,),name="Dec_states")
    
    encoder_outputs = Input(shape=(ENCODER_SEQ_LEN, units,),name="EncoderStates")
        
    #mask_encoder = Input(shape=(ENCODER_SEQ_LEN,), name='mask_encoder')
    
    # Embedding layers
    dec_embdd = Embedding(input_dim=VOCAB_SIZE, output_dim=50,
                      input_length=1,mask_zero=False, name="Decoder_Embedding_layer")(input_decoder)
    
    
    z_attention = dict()
    z_attention['enocderHs'] = encoder_outputs
    z_attention['decoderHt'] = input_state
    
    context_vector = clayers.BahdanauAttention(units=20, name='Attention')(z_attention)
    
    concat = concatenate([tf.cast(tf.expand_dims(context_vector, 1), dec_embdd.dtype), dec_embdd],name="concat")
    
    decoder_output, Decoder_state = GRU(units=units,return_state=True,name="DecGRU")(concat, initial_state=input_state)
    
    output = Dense(units=VOCAB_SIZE,activation="softmax",name="DenseOut")(decoder_output)
    
    return Model(inputs=[input_decoder, input_state, encoder_outputs],outputs=[output, Decoder_state],name="OneStepDecoder")

In [9]:
def encoder_decoder():
    
    encoder_input = Input(shape=(ENCODER_SEQ_LEN,), name='encoder_input_final')
    decoder_input = Input(shape=(DECODER_SEQ_LEN,), name="Decoder_inout_final")
    
    enc = encoder()

    outputs_encoder, encoder_state = enc(encoder_input)
    
    decoder_one_att = OneStepDecoder()
    
    all_outputs= []
     
    
    for timestep in range(DECODER_SEQ_LEN):
        
        inputs = Lambda(lambda x: x[:,timestep:timestep+1])(decoder_input)
        
        output, encoder_state = decoder_one_att([inputs, encoder_state, outputs_encoder])
        
        output = Lambda(lambda x:  tf.expand_dims(x, 1))(output)
        
        all_outputs.append(output)
        
        
    decoder_outputs = Lambda(lambda x: tf.keras.backend.concatenate(all_outputs,1))(all_outputs)
    
    return Model(inputs=[encoder_input, decoder_input], outputs=decoder_outputs)

In [10]:
EC = encoder_decoder()

W1023 15:23:10.052164 11088 deprecation.py:506] From D:\Softwares\Anaconda3\lib\site-packages\tensorflow\python\keras\initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1023 15:23:10.112370 11088 deprecation.py:506] From D:\Softwares\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
EC.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder_inout_final (InputLayer [(None, 20)]         0                                            
__________________________________________________________________________________________________
encoder_input_final (InputLayer [(None, 30)]         0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1)            0           Decoder_inout_final[0][0]        
__________________________________________________________________________________________________
model (Model)                   [(None, 30, 16), (No 28216       encoder_input_final[0][0]        
____________________________________________________________________________________________

In [12]:
EC.inputs

[<tf.Tensor 'encoder_input_final:0' shape=(?, 30) dtype=float32>,
 <tf.Tensor 'Decoder_inout_final:0' shape=(?, 20) dtype=float32>]

In [13]:
EC.outputs

[<tf.Tensor 'lambda_40/concat:0' shape=(?, 20, 500) dtype=float32>]

In [14]:
EC.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.sparse_categorical_crossentropy)

In [15]:
EC.fit(x=[x,y],y=y,epochs=1,verbose=1,batch_size=32)

W1023 15:23:33.241009 11088 deprecation.py:323] From D:\Softwares\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2000/2000 [==============================] - 21s 11ms/sample - loss: 6.1863
